# Static Analyzer - QA Assignment 
Team 12
* Reema Khaseeb - 
* Yara Abu-Odeh - 11926410
* Zubaida Sadder - 11925174 <br>

Doc. Mustafa Assaf <br>

### Generally needed functions. 

In [1]:
def read_func(filename):
    '''
    Return: the text file in a list of its lines.
    Parameter: (filename) should be .txt format.
    Description: It reads the text file that includes the code line by line.
    ''' 
    with open(filename,'r') as file:
        lines = [line.strip() for line in file]

    return lines

In [2]:
def functions_lines(lines_list):
    '''
    return: List of each line with function definition
    parameter: list of lines 
    ''' 
    return [index+1 for index,line in enumerate(lines_list)
            if len(line.split()) >= 1 
            if line.split()[0] == 'def']

In [3]:
def find (keyword,filename,what):
    '''
    Return: List of words or list of lines that includes a certain word. 
    Parameter: (keyword: The word you're searching for,
                filename: The text file, 
                what: "word" to return a list of words or "lines" list lines) 
    Description: its used when you want to find a word or the line containing the word
    '''
    store = []
    with open(filename) as f:  
        for line in f:      
            for word in line.split():
                if word == keyword and what == "word":
                    store.append(word)
                if word == keyword and what == "line":
                    store.append(line)
    return store

In [4]:
def split_by(sentence, by):
    '''
    Return: List of splitted sentence
    Parameter: (sentence: The sentence to split,
                by: to split by what)
    Description: Splits strings using split method 
    '''
    splitted = sentence.split(by)
    return splitted

### Magic Number 

In [67]:
def magic_num(lines_list):
    '''
    Return: Print an error msg if there's a magic number. 
    Parameter: (lines_list: the list that stores the text file)
    Description: Checks the occurance of a magic number considering that it happens 
                after a list of operations or keywords.
    '''
    operations = ['>', '<', '==','!=', '**','+', '-', '/','*', 'return']
    for i in range(len(lines_list)):
             for j in range(len(lines_list[i])):
                if len(lines_list[i]) != j+1:
                    isNumber =  lines_list[i][j+1].isnumeric()
                    for operation in operations:
                        if (operation == lines_list[i][j] and isNumber) :
                            print(f'[WARNING]: Magic Number. Line {i} Where:  " {lines_list[i]} "')

### Division by zero 

In [33]:
def Line_OfDivid(lines_list):
    '''
    Return: List of lines that contains division operations 
    Parameter: (lines_list: list of lines from the text file)
    Description: Search for '/' string in each line of the file.
    '''
    LineOfDivid =[]
    for i in range(len(lines_list)):
        for j in lines_list[i]:
            if j=="/":
                LineOfDivid.append(lines_list[i])
    return LineOfDivid
   

In [39]:
def AfterDiv(lines_list):
    '''
    Return: List of values (variables or numbers) after each division
    Parameter: (lines_list: list of lines from the text file)
    Description: Search for the values after the divison
    '''
    LD = Line_OfDivid(lines_list)
    list_num = []
    for i in range(len(LD)):
        num = LD[i].split("/")
        list_num.append(num[1])
    return list_num

In [40]:
def DividByZero(filename,lines_list):
    '''
    Return: Print if there exist division by zero
    Parameter: (filename: The name of the text file that contains the code.
                lines_list: list of lines from the text file)
    Description: Checks if the number after the division is zero   
                and check the value of the variable if is zero or not.
    '''
    List_num=AfterDiv(lines_list)
    for i in range(len(List_num)):
        if List_num[i].isnumeric() == True:
            if int(List_num[i]) == 0:
                 print("[ERROR] : Division by zero")            
        else: 
            res = []
            for i in List_num:
                var_div = find(i, filename, "line")
            for i in range(len(var_div)):
                num = var_div[i].split("=")[1]
                res.append(int(num))
                if res == 0: 
                    print("[ERROR] : Division by zero")

### No more than three parameters

In [9]:
def get_parameters(func):
    '''
    Return: List of parameter
    Parameter: (func: string; the parameters you want to make them in a list) 
    Description: Splits the string to get the exact parameters between the brackets.  
    '''
    #Split between whats before and after the ( ) .
    obracket = split_by(func,"(")
    cbracket = split_by(obracket[1],")")
    #result is a list of parameters 
    result = split_by(cbracket[0],",")
    return result


In [10]:
def len_parameters(pars,location):
    '''
    Return: Print the error msg if there's more than three parameters 
    Parameter: (pars: list of parameters  , location: the function that contains the parameters) 
    Description: Check if number of parameters is 3 or more in a certain line. 
    '''
    number_parameters = len(pars)
    max_parameters = 3 
    if number_parameters>max_parameters :
        print(f"[ERROR]: More than three parameters. There's {number_parameters} parameters in function:{location}")

In [11]:
def parameters_number(filename):
    '''
    Return: Print the error msg if there's more than three parameters 
    Parameter: (filename: the text file that contains the code) 
    Description: Check if number of parameters is 3 or more on the whole code (text file). 
    '''
    data = read_func(filename)
    #case 1: function is after the definition
    functions = find("def",filename,"line")
    for i in range(len(functions)):
        x = get_parameters(split_by(functions[i], " ")[1])
        len_parameters(x,functions[i])
    #case 2: function is called in the code.
    names = []
    for i in functions:
        names.append(split_by(i, " ")[1])
    for i in names:
        if i in data:
            x = get_parameters(i)
            len_parameters(x,i)

### Unreachable Code

Case 1: False condition or True condition

In [64]:
lines_list = read_func("test.txt")
def_lines = functions_lines(lines_list)

In [49]:
def get_next_function_line(index):
    '''
    Loop from that index line until the next function
    return: next function line
    parameter: index line
    '''
    for def_line in def_lines:
        if def_line > index:
            return def_line

In [50]:
def false_true_conditions(lines_list):
    for index_line,line in enumerate(lines_list):
        if 'if' in line:
            if 'False' in line:
                print(f'[WARNING]: Unrechable code in line {index_line+2}, False condition')

            elif 'True' in line:
                detect_until = get_next_function_line(index_line+1)

                for line_else_index,lines_until_def in enumerate(lines_list[index_line:detect_until]):
                    if 'else' in lines_until_def:
                        print(f'[WARNING]: Unrechable code in line {line_else_index+2}, if statement always true, therefore else condition is unreachable')

Case 2: After return statement

In [51]:
def after_return_statement(lines_list):
    for index,line in enumerate(lines_list):
        if 'return' in line:
            next_line = lines_list[index+1].split()
            if len(next_line) >= 1: # not empty line
                # if the next line is neither 'else' statement nor new function
                if 'else' in next_line or 'def' in next_line:
                    pass
                else:
                    print(f'[WARNING]: Unreachable code after return statement in line {index+2}')

Case 3: Same condition in if and else statement

In [52]:
import re

In [53]:
def save_condition(line, statement):
    cond = re.split(':',line)[0]
    cond = re.split(statement, cond)[1].strip()
    return cond

In [54]:
def duplicated_condition(lines_list):
    for index,line in enumerate(lines_list):
        if 'if' in line:
            condition = save_condition(line, 'if')
        # if not empty line
        if len(line) >= 1 and 'elif' in line: 
                elif_condition = save_condition(line, 'elif')
                # Detect if else condition is the same as elif condition
                if elif_condition == condition:
                    print(f'[WARNING]: Unreachable code at line {index+2},the elif-statement can never be true as the same condition in if-statement has already been handled')

In [55]:
def unreachable_code(lines_list):
    false_true_conditions(lines_list)
    after_return_statement(lines_list)
    duplicated_condition(lines_list)

In [65]:
def main(filename):
    '''
    Calls the four cases. 
    '''
    magic_num(lines_list)
    DividByZero(filename,lines_list)
    parameters_number(filename)
    unreachable_code(lines_list)

In [66]:
main("test.txt")

[Error]: Magic Error. Line 18 Where:  " if x>2: "
[Error]: Magic Error. Line 33 Where:  " q = x/0 "
[Error]: Magic Error. Line 34 Where:  " f = x+10 "
[Error]: Magic Error. Line 40 Where:  " for i>7: "
[ERROR] : Division by zero
[ERROR]: More than three parameters. There's 5 parameters in function:def check1(x,n,z,w,q) :

[ERROR]: More than three parameters. There's 5 parameters in function:check1(x,n,z,w,q)
[WARNING]: Unrechable code in line 4, if statement always true, therefore else condition is unreachable
[WARNING]: Unrechable code in line 9, False condition
[WARNING]: Unreachable code after return statement in line 16
[WARNING]: Unreachable code after return statement in line 21
[WARNING]: Unreachable code at line 30,the elif-statement can never be true as the same condition in if-statement has already been handled
